In [2]:
!source ../meta_env/bin/activate

In [3]:
!which python

/util/common/python/ondemand-jupyter/py38/anaconda-2021.05/bin/python


In [6]:
!pip install torch torchvision matplotlib tqdm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 619.9 MB 42 kB/s s eta 0:00:01  |                                | 102 kB 5.8 MB/s eta 0:01:48     |▎                               | 5.0 MB 5.8 MB/s eta 0:01:47     |▎                               | 5.6 MB 5.8 MB/s eta 0:01:47     |▉                               | 15.4 MB 8.4 MB/s eta 0:01:12     |█                               | 21.5 MB 10.9 MB/s eta 0:00:55     |█▍                              | 25.9 MB 10.9 MB/s eta 0:00:55     |█▉                              | 35.8 MB 8.5 MB/s eta 0:01:10     |██                              | 39.7 MB 8.5 MB/s eta 0:01:09     |██▍                             | 47.0 MB 11.6 MB/s eta 0:00:50     |██▊                             | 53.1 MB 11.6 MB/s eta 0:00:49     |███▊                            | 71.3 MB 8.4 MB/s eta 0:01:06     |███▊                            | 71.9 MB 8.4 MB/s eta 0:01:06     |███▊                            

     |████████████████████████████████| 33.8 MB 11.1 MB/s eta 0:00:01    |████████▍                       | 8.9 MB 45.4 MB/s eta 0:00:01     |█████████████████               | 18.0 MB 11.0 MB/s eta 0:00:02     |███████████████████▎            | 20.3 MB 11.0 MB/s eta 0:00:02     |████████████████████████▉       | 26.2 MB 8.8 MB/s eta 0:00:01     |███████████████████████████▌    | 29.1 MB 8.8 MB/s eta 0:00:01     |█████████████████████████████▊  | 31.4 MB 8.8 MB/s eta 0:00:01     |██████████████████████████████▎ | 32.0 MB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 63.2 MB 11.1 MB/s eta 0:00:01    |██▌                             | 5.0 MB 45.7 MB/s eta 0:00:02     |████████████████▍               | 32.3 MB 8.0 MB/s eta 0:00:04     |███████████████████▌            | 38.6 MB 10.9 MB/s eta 0:00:03     |███████████████████████         | 45.6 MB 11.0 MB/s eta 0:00:02     |███████████████████████▊        | 46.

     |████████████████████████████████| 168.4 MB 10.2 MB/s eta 0:00:01   |▎                               | 1.3 MB 64.9 MB/s eta 0:00:03     |█▏                              | 5.9 MB 64.9 MB/s eta 0:00:03     |█▎                              | 6.5 MB 64.9 MB/s eta 0:00:03     |███▏                            | 16.8 MB 12.7 MB/s eta 0:00:12     |███▉                            | 20.2 MB 12.7 MB/s eta 0:00:12     |█████▎                          | 27.7 MB 9.2 MB/s eta 0:00:16     |████████                        | 42.1 MB 14.5 MB/s eta 0:00:09     |████████▏                       | 42.7 MB 14.5 MB/s eta 0:00:09     |████████▎                       | 43.3 MB 14.5 MB/s eta 0:00:09     |████████▊                       | 46.2 MB 11.1 MB/s eta 0:00:11     |█████████                       | 47.4 MB 11.1 MB/s eta 0:00:11     |██████████▍                     | 54.9 MB 11.1 MB/s eta 0:00:11     |███████████▎                    | 59.6 MB 11.1 MB/s eta 0:00:10     |████████████                    |

In [4]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

class MyTransform:
    def __init__(self, image_size, horizontal_flip=False, rotation_angle=None):
        transform_list = [
            #transforms.RandomResizedCrop(image_size),
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]
        if horizontal_flip:
            transform_list.append(transforms.RandomHorizontalFlip())
        if rotation_angle is not None:
            transform_list.append(transforms.RandomRotation(degrees=rotation_angle))

        self.transform = transforms.Compose(transform_list)

    def __call__(self, img):
        return self.transform(img)

# Path to the ImageNet dataset directory
#data_dir = './ImageNet'

# Parameters for image augmentation
image_size = 224
horizontal_flip = False
rotation_angle = None

In [5]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
import numpy as np
import torch

# define configuration
class Config:
    def __init__(self):
        self.n_way = 3  # N-way
        self.k_spt = 5  # K-shot
        self.k_query = self.k_qry =10  # K-query
        self.img_size = (224, 224)  # size of images
        self.batch_size = 1  # batch size
        self.inner_update_lr = 0.01  # inner update learning rate
        self.meta_lr = 1e-3  # meta learning rate
        self.meta_batch_size = 10  # size of meta batch
        self.max_epoch = 10  # number of epochs
        self.num_tasks = 10  # number of tasks

config = Config()



In [37]:
import os
import random
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

try:
    !rm -r /user/asugam/meta_learning_project/data/meta_tasks_symlink/
except:
    print("nothing to delete")
finally:
    !mkdir /user/asugam/meta_learning_project/data/meta_tasks_symlink
    
    
class MetaLearningDataset(Dataset):
    def __init__(self, root_dir, num_tasks, num_classes, support_set_size, query_set_size,transform = None, seed = 0):
        self.num_tasks = num_tasks
        self.num_classes = num_classes
        self.support_set_size = support_set_size
        self.query_set_size = query_set_size
        self.seed = seed
        self.transform = transform

        self.root_dir = root_dir
        self.task_dir = '/user/asugam/meta_learning_project/data/meta_tasks_symlink'
        self.class_names = os.listdir(self.root_dir)
        
        self.task_seeds = np.arange(seed, seed + num_tasks)
        np.random.shuffle(self.task_seeds)

        # Create task directories
        for task_seed in self.task_seeds:
            task_seed = int(task_seed)
            random.seed(task_seed)
            np.random.seed(task_seed)

            task_classes = random.sample(self.class_names, self.num_classes)
            print(f'### {task_classes}')
            # Create symbolic links to the chosen class directories in the task directory
            for class_name in task_classes:
                class_dir = os.path.join(self.root_dir, class_name)
                class_images = os.listdir(class_dir)
                task_images = random.sample(class_images, self.support_set_size + self.query_set_size)
                for img_name in task_images:
                    src = os.path.join(class_dir, img_name)
                    dst_dir = os.path.join(self.task_dir, f'task_{task_seed}', class_name)
                    os.makedirs(dst_dir, exist_ok=True)
                    dst = os.path.join(dst_dir, img_name)
                    os.symlink(src, dst)
        print('created symlinks for each task')

    def __getitem__(self, index):
        task_seed = int(self.task_seeds[index%self.num_tasks])
        print(task_seed, self.task_seeds)
        task_folder = os.path.join(self.task_dir, f'task_{task_seed}')
        
        # Load the task directory using ImageFolder
        print(os.listdir(task_folder))
        image_dataset = ImageFolder(task_folder, transform=self.transform)

        # Randomly select the support and query set
        random.seed(task_seed)
        np.random.seed(task_seed)

        support_images = []
        support_labels = []
        query_images = []
        query_labels = []

        for class_idx in range(self.num_classes):
            # Get all images of this class
            class_images = [(img, target) for img, target in image_dataset if target == class_idx]
            
            # Ensure there are enough images for support and query set
            assert len(class_images) >= self.support_set_size + self.query_set_size

            # Randomly select images for support set and query set
            indices = list(range(len(class_images)))
            random.shuffle(indices)

            # Select support_set_size images for support set
            for i in indices[:self.support_set_size]:
                img, label = class_images[i]
                support_images.append(img)
                support_labels.append(torch.tensor(class_idx))

            for i in indices[self.support_set_size:self.support_set_size + self.query_set_size]:
                img, label = class_images[i]
                query_images.append(img)
                query_labels.append(torch.tensor(class_idx))
                
        # Shuffle support set and query set
        support_set = list(zip(support_images, support_labels))
        random.shuffle(support_set)
        support_images, support_labels = zip(*support_set)

        query_set = list(zip(query_images, query_labels))
        random.shuffle(query_set)
        query_images, query_labels = zip(*query_set)

        # Convert lists of images and labels to tensors
        support_set_X_batch = torch.stack(support_images)
        support_set_y_batch = torch.stack(support_labels)
        query_set_X_batch = torch.stack(query_images)
        query_set_y_batch = torch.stack(query_labels)

        return support_set_X_batch, support_set_y_batch, query_set_X_batch, query_set_y_batch

    def __len__(self):
        return self.num_tasks

In [38]:
train_dataset_path = '/user/asugam/meta_learning_project/data/big_final_category' 

In [39]:
meta_learning_loader = MetaLearningDataset(root_dir = train_dataset_path , num_tasks = 5, num_classes = 3, support_set_size = 5, query_set_size = 10,transform =MyTransform(image_size, horizontal_flip, rotation_angle) )

### ['Atelectasis', 'Hernia', 'Edema']
### ['Atelectasis', 'Edema', 'Hernia']
### ['Atelectasis', 'Hernia', 'Edema']
### ['Edema', 'Hernia', 'Atelectasis']
### ['Atelectasis', 'Hernia', 'Edema']
created symlinks for each task


In [46]:
val = meta_learning_loader[7]
val[0].shape

1 [3 4 1 0 2]
['Atelectasis', 'Edema', 'Hernia']


torch.Size([15, 3, 224, 224])

IndexError: tuple index out of range

In [22]:
import torch
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
from copy import deepcopy
import torchvision.models as models

class MyAlexNet(nn.Module):
    def __init__(self, num_classes=3, use_dropout=False, reg_strength = 0.2):
        super(MyAlexNet, self).__init__()

        self.reg_strength = reg_strength

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Dropout() if use_dropout else nn.Identity(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout() if use_dropout else nn.Identity(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def l1_regularization_loss(self):
        l1_loss = torch.tensor(0.0).to(self.features[0].weight.device)
        for param in self.parameters():
            l1_loss += torch.norm(param, p=1)
        return self.reg_strength * l1_loss

    def l2_regularization_loss(self):
        l2_loss = torch.tensor(0.0).to(self.features[0].weight.device)
        for param in self.parameters():
            l2_loss += torch.norm(param, p=2)
        return self.reg_strength * l2_loss

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

class MyResNet(torch.nn.Module):
    def __init__(self, num_classes=3):
        super(MyResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = torch.nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)